In [1]:
import os
import json
import datetime
import time

from dateutil import parser
import glob
import json
from dataclasses import dataclass, field
from dacite import from_dict, Config
from typing import Optional
import pandas as pd

CREDENTIALS_FILE = "../creds.txt"

# Load Twitter API OAuth and other details

from TwitterAPI import TwitterAPI, TwitterOAuth, TwitterRequestError

o = TwitterOAuth.read_file(CREDENTIALS_FILE)
api = TwitterAPI(o.consumer_key, o.consumer_secret, o.access_token_key, o.access_token_secret, auth_type="oAuth2", api_version="2")


# directory = "../datasets/users/ia"
# filename = "users-ia.json"
# filepath = os.path.join(directory, filename)


In [2]:
# user_ids = set()
# with open(filepath, "r") as f:
#     for line in f:
#         user_ids.add(json.loads(line.strip())["id"])
df = pd.read_pickle("timeline_tweets.pkl")

In [5]:
users = df["author_id"].unique()
len(users)

1393

In [6]:
def request_with_timeout(endpoint, params):
    try:
        result = api.request(endpoint, params)
    except TwitterRequestError as e:
        print(e)
        
    return result
    
until = datetime.date(2022,1,1)
n_requests = 0

for i, user in enumerate(users):
    print(f"User {i}/{len(users)}: {user}")
    times = df[df["author_id"] == user]["created_at"]
    latest_tweet = times.max() # print(f"From {times.min()} to {times.max()}")
    if latest_tweet > "2021-07-01":
        continue
    params = {
        "tweet.fields": "id,text,author_id,created_at,withheld",
        "expansions": "geo.place_id", #,referenced_tweets.id",
        "max_results": 500,
        "start_time": latest_tweet,
        "end_time": until.strftime('%Y-%m-%dT%H:%M:%SZ'),
        "query": f"from:{user}"
    }
    
    with open("timelines.json", "a+") as f:
        n_tweets = 0
        while True:
            n_requests += 1
            if n_requests >= 300:
                print("Sleeping 15 minutes for ")
                time.sleep(60*15)
                n_requests = 0
            time.sleep(1)
            tweets = request_with_timeout("tweets/search/all", params)
            for t in tweets:
                n_tweets += 1
                json.dump(t, f)
                f.write("\n")
            meta = tweets.json()["meta"]
            if "next_token" in meta:
                next_token = meta["next_token"]
                params["next_token"] = next_token
            else:
                break
        print(f"... pulled {n_tweets} tweets for {user} from {latest_tweet} until {str(until)}")

User 0/1393: 731468792295882752
User 1/1393: 1096814475372462081
User 2/1393: 736185001457287169
User 3/1393: 1031570363577073666
... pulled 1 tweets for 1031570363577073666 from 2019-11-05T01:16:41.000Z until 2022-01-01
User 4/1393: 889809481453797377
... pulled 1 tweets for 889809481453797377 from 2021-05-01T02:09:23.000Z until 2022-01-01
User 5/1393: 887570995539607555
User 6/1393: 795049962295787522
User 7/1393: 891000197039259650
User 8/1393: 878493010773671940
... pulled 1 tweets for 878493010773671940 from 2020-09-24T08:24:31.000Z until 2022-01-01
User 9/1393: 850003218704629764
User 10/1393: 743856625631395847
User 11/1393: 96763916
User 12/1393: 2595946512
User 13/1393: 1044987528317730827
User 14/1393: 3192389641
User 15/1393: 894525472062689281
User 16/1393: 183959572
User 17/1393: 987037716
... pulled 1 tweets for 987037716 from 2021-01-11T17:32:48.000Z until 2022-01-01
User 18/1393: 1340211230
User 19/1393: 3282141216
User 20/1393: 1007631212385525760
User 21/1393: 1128731

User 160/1393: 1316946288
User 161/1393: 950609618455085056
... pulled 1 tweets for 950609618455085056 from 2020-07-14T22:26:10.000Z until 2022-01-01
User 162/1393: 3958178172
User 163/1393: 3824255359
... pulled 1 tweets for 3824255359 from 2019-12-15T10:39:24.000Z until 2022-01-01
User 164/1393: 303661441
User 165/1393: 2451603842
User 166/1393: 1081145078754664448
User 167/1393: 1075428083220439040
... pulled 1 tweets for 1075428083220439040 from 2020-03-22T17:47:03.000Z until 2022-01-01
User 168/1393: 1443496322
User 169/1393: 115409287
User 170/1393: 23462280
... pulled 1 tweets for 23462280 from 2021-02-14T10:37:23.000Z until 2022-01-01
User 171/1393: 1048981
User 172/1393: 879999110417235968
User 173/1393: 739744952800792576
User 174/1393: 1125387161132355584
User 175/1393: 1061076300318658560
User 176/1393: 1121354180499623936
User 177/1393: 3013542302
User 178/1393: 705289694293020672
User 179/1393: 1047853414535680000
User 180/1393: 398098844
User 181/1393: 109988070451762380

... pulled 94 tweets for 555876917 from 2021-06-24T16:29:06.000Z until 2022-01-01
User 247/1393: 860904751206023168
... pulled 1 tweets for 860904751206023168 from 2019-02-11T22:27:55.000Z until 2022-01-01
User 248/1393: 1110934074
... pulled 1 tweets for 1110934074 from 2021-04-21T12:08:06.000Z until 2022-01-01
User 249/1393: 140214844
... pulled 1212 tweets for 140214844 from 2021-06-30T05:31:50.000Z until 2022-01-01
User 250/1393: 427655745
... pulled 4248 tweets for 427655745 from 2021-06-30T23:51:21.000Z until 2022-01-01
User 251/1393: 33325635
... pulled 1 tweets for 33325635 from 2020-12-17T10:45:11.000Z until 2022-01-01
User 252/1393: 3196813892
... pulled 358 tweets for 3196813892 from 2021-06-29T23:24:42.000Z until 2022-01-01
User 253/1393: 3557229132
... pulled 7915 tweets for 3557229132 from 2021-06-30T23:28:59.000Z until 2022-01-01
User 254/1393: 948444899082293248
... pulled 8 tweets for 948444899082293248 from 2021-06-26T07:38:09.000Z until 2022-01-01
User 255/1393: 7796

... pulled 1 tweets for 1162698710406459392 from 2020-08-28T14:16:26.000Z until 2022-01-01
User 319/1393: 1131146060657651712
... pulled 2 tweets for 1131146060657651712 from 2021-02-17T15:39:26.000Z until 2022-01-01
User 320/1393: 1055621880
... pulled 22 tweets for 1055621880 from 2021-05-29T15:07:26.000Z until 2022-01-01
User 321/1393: 941371451038351360
... pulled 29 tweets for 941371451038351360 from 2021-06-19T08:41:38.000Z until 2022-01-01
User 322/1393: 1112608298908315648
... pulled 1 tweets for 1112608298908315648 from 2020-06-10T11:57:17.000Z until 2022-01-01
User 323/1393: 985375291538206720
... pulled 6 tweets for 985375291538206720 from 2021-05-29T16:19:30.000Z until 2022-01-01
User 324/1393: 133726970
... pulled 53 tweets for 133726970 from 2021-06-08T14:51:27.000Z until 2022-01-01
User 325/1393: 2336624767
... pulled 43 tweets for 2336624767 from 2021-06-30T11:45:51.000Z until 2022-01-01
User 326/1393: 1743061760
... pulled 1 tweets for 1743061760 from 2021-01-09T23:28:

TwitterRequestError: ('{"title":"Too Many Requests","detail":"Too Many Requests","type":"about:blank","status":429}',) (429): {"title":"Too Many Requests","detail":"Too Many Requests","type":"about:blank","status":429}